# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 08

Group Members:
- Daffari Adiyatma (18222003)
- Aththariq Lisan Q. D. S. (18222013)
- Muhammaf Rafly (18222067)
- Alvin Fadhillah Akmal (18222079)

## Import Libraries

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE

## Import Dataset

In [12]:
if 'df_original' not in globals():
    df_original = pd.read_csv('https://drive.google.com/uc?id=1w8DnxXGwaF1dLKlMxmXWVQFOH4vcvfTp')

df = df_original.copy()
df.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [13]:
#Drop kolom yang tidak digunakan
df = df.drop(columns=['FILENAME'])

In [14]:
df_sample = df.sample(frac=0.2, random_state=42)
df_sample.shape

(28081, 55)

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [15]:
train_set, val_set = train_test_split(df_sample, test_size=0.2, random_state=42, shuffle=True)

print(f"Ukuran training set: {len(train_set)}")
print(f"Ukuran validation set: {len(val_set)}")

Ukuran training set: 22464
Ukuran validation set: 5617


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [16]:
train_duplicates = train_set.duplicated().sum()
print(f"Jumlah duplikasi pada train_set: {train_duplicates}")

val_duplicates = val_set.duplicated().sum()
print(f"Jumlah duplikasi pada val_set: {val_duplicates}")

Jumlah duplikasi pada train_set: 0
Jumlah duplikasi pada val_set: 0


Tidak ada duplikasi sehingga proses cukup sampai dipengecekkan. Remove duplikasi dipindahkan ke urutan teratas karena mudah untuk mengecekknya.

In [17]:
class RemoveDuplicatesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        duplicates = X.duplicated().sum()
        if duplicates > 0:
            print(f"Jumlah duplikasi: {duplicates}. Duplikasi dihapus.")
            return X.drop_duplicates()
        else:
            print("Tidak ada duplikasi ditemukan.")
            return X

### II. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [18]:
#Memeriksa Missing Value
missing_count = train_set.isnull().sum()

missing_percentage = (missing_count / len(train_set)) * 100

missing_data = pd.DataFrame({
    'Missing Count': missing_count,
    'Percentage': missing_percentage
})

missing_data_sorted = missing_data.sort_values(by='Missing Count', ascending=False)

print(missing_data_sorted[missing_data_sorted['Missing Count'] > 0])


                            Missing Count  Percentage
Domain                              11263   50.137999
LineOfCode                          11105   49.434651
NoOfExternalRef                     11098   49.403490
HasSocialNet                        10871   48.392984
NoOfCSS                             10868   48.379630
LargestLineLength                   10798   48.068020
HasCopyrightInfo                    10756   47.881054
NoOfURLRedirect                     10710   47.676282
NoOfObfuscatedChar                  10617   47.262286
HasObfuscation                      10605   47.208868
NoOfSelfRedirect                    10560   47.008547
HasPasswordField                    10550   46.964031
LetterRatioInURL                    10470   46.607906
ObfuscationRatio                    10316   45.922365
NoOfLettersInURL                    10184   45.334758
SpacialCharRatioInURL               10056   44.764957
HasSubmitButton                     10003   44.529024
NoOfEqualsInURL             

In [19]:
#Drop Kolom yang missing valuenya di atas 49%
columns_to_drop = ['LineOfCode', 'NoOfExternalRef', 'Domain']

train_set.drop(columns=columns_to_drop, axis=1, inplace=True)
val_set.drop(columns=columns_to_drop, axis=1, inplace=True)

print("Columns dropped:", columns_to_drop)

Columns dropped: ['LineOfCode', 'NoOfExternalRef', 'Domain']


In [20]:
class ImputationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.binary_imputer = SimpleImputer(strategy='most_frequent')
        self.mean_imputer = SimpleImputer(strategy='mean')
        self.median_imputer = SimpleImputer(strategy='median')
        self.binary_columns = []
        self.mean_columns = []
        self.median_columns = []

    def fit(self, X, y=None):
        numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns
        
        self.binary_columns = [col for col in numerical_columns if set(X[col].dropna().unique()) <= {0, 1}]
        
        non_binary_columns = [col for col in numerical_columns if col not in self.binary_columns]
        skewness = X[non_binary_columns].skew()
        self.mean_columns = skewness[skewness.abs() < 0.5].index.tolist()
        self.median_columns = skewness[skewness.abs() >= 0.5].index.tolist()
        
        if self.binary_columns:
            self.binary_imputer.fit(X[self.binary_columns])
        if self.mean_columns:
            self.mean_imputer.fit(X[self.mean_columns])
        if self.median_columns:
            self.median_imputer.fit(X[self.median_columns])

        return self

    def transform(self, X):
        X = X.copy()
        
        if self.binary_columns:
            X[self.binary_columns] = self.binary_imputer.transform(X[self.binary_columns])
        
        if self.mean_columns:
            X[self.mean_columns] = self.mean_imputer.transform(X[self.mean_columns])
        
        if self.median_columns:
            X[self.median_columns] = self.median_imputer.transform(X[self.median_columns])
        
        return X


In [21]:
#Drop kolom kategorikal karena tidak akan diimputasi
categorical_columns = train_set.select_dtypes(include=['object']).columns

train_set.drop(columns=categorical_columns, axis=1, inplace=True)
val_set.drop(columns=categorical_columns, axis=1, inplace=True)

print("Columns dropped (categorical):", categorical_columns.tolist())

Columns dropped (categorical): ['URL', 'TLD', 'Title']


### III. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [22]:
numerical_columns = train_set.select_dtypes(include=['float64', 'int64']).columns
binary_columns = [col for col in numerical_columns if set(train_set[col].dropna().unique()) <= {0, 1}]

parameters = {}

print("Outlier Sebelum Imputasi:")
for col in numerical_columns:
    if col in binary_columns:
        continue  
    
    Q1 = train_set[col].quantile(0.25)
    Q3 = train_set[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_below = (train_set[col] < lower_bound).sum()
    outliers_above = (train_set[col] > upper_bound).sum()
    
    print(f"{col}:")
    print(f"  Outlier di bawah batas: {outliers_below}")
    print(f"  Outlier di atas batas: {outliers_above}")

    parameters[col] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}

Outlier Sebelum Imputasi:
id:
  Outlier di bawah batas: 0
  Outlier di atas batas: 0
URLLength:
  Outlier di bawah batas: 0
  Outlier di atas batas: 428
DomainLength:
  Outlier di bawah batas: 8
  Outlier di atas batas: 301
CharContinuationRate:
  Outlier di bawah batas: 2338
  Outlier di atas batas: 0
TLDLegitimateProb:
  Outlier di bawah batas: 0
  Outlier di atas batas: 0
URLCharProb:
  Outlier di bawah batas: 539
  Outlier di atas batas: 45
TLDLength:
  Outlier di bawah batas: 0
  Outlier di atas batas: 77
NoOfSubDomain:
  Outlier di bawah batas: 167
  Outlier di atas batas: 2425
NoOfObfuscatedChar:
  Outlier di bawah batas: 0
  Outlier di atas batas: 4
ObfuscationRatio:
  Outlier di bawah batas: 0
  Outlier di atas batas: 5
NoOfLettersInURL:
  Outlier di bawah batas: 0
  Outlier di atas batas: 347
LetterRatioInURL:
  Outlier di bawah batas: 38
  Outlier di atas batas: 39
NoOfDegitsInURL:
  Outlier di bawah batas: 0
  Outlier di atas batas: 824
DegitRatioInURL:
  Outlier di bawah b

In [23]:
class OutlierHandlerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.parameters = {}
        self.binary_columns = []

    def fit(self, X, y=None):
        numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns
        
        self.binary_columns = [col for col in numerical_columns if set(X[col].dropna().unique()) <= {0, 1}]
        non_binary_columns = [col for col in numerical_columns if col not in self.binary_columns]
        
        self.parameters = {}
        for col in non_binary_columns:
            q1 = X[col].quantile(0.25)
            q3 = X[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            self.parameters[col] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}
        return self

    def transform(self, X):
        X = X.copy()
        
        for col, bounds in self.parameters.items():
            X[col] = X[col].clip(lower=bounds['lower_bound'], upper=bounds['upper_bound'])
        
        return X

Sudah lebih baik untuk persebaran dan nilai-nilai ouliernya, meskipun metode clipping membuat penumpukan data di nilai min dan max

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [24]:
class UnifiedFeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, important_features=None, low_corr_threshold=0.2, high_corr_threshold=0.9, bins=5):
        self.important_features = important_features if important_features else set()
        self.low_corr_threshold = low_corr_threshold
        self.high_corr_threshold = high_corr_threshold
        self.bins = bins
        self.features_to_drop = set()
        self.interaction_features = []
        self.columns = None  # Untuk menyimpan kolom final

    def fit(self, X, y=None):
        # Identifikasi kolom yang konstan
        self.features_to_drop = set(X.nunique()[X.nunique() <= 1].index)

        numerical_columns = X.select_dtypes(include=['float64', 'int64'])
        
        if y is not None:
            # Hitung korelasi dengan target dan tambahkan kolom berkorelasi rendah ke fitur yang akan dihapus
            correlation_with_target = numerical_columns.corrwith(y).abs()
            low_correlation_features = set(correlation_with_target[correlation_with_target < self.low_corr_threshold].index)
            self.features_to_drop.update(low_correlation_features.difference(self.important_features))

        # Identifikasi fitur dengan korelasi tinggi antar fitur
        corr_matrix = numerical_columns.corr().abs()
        for col in numerical_columns.columns:
            highly_correlated = corr_matrix.index[(corr_matrix[col] > self.high_corr_threshold) & (corr_matrix.index != col)]
            self.interaction_features.extend([(col, other) for other in highly_correlated if (other, col) not in self.interaction_features])

        # Simpan kolom yang ada setelah fitting
        self.columns = list(X.drop(columns=self.features_to_drop, errors='ignore').columns)
        return self

    def transform(self, X):
        X = X.copy()
        
        # Hapus fitur yang tidak relevan
        X = X.drop(columns=self.features_to_drop, errors='ignore')

        # Tambahkan fitur interaksi
        for feature_1, feature_2 in self.interaction_features:
            if feature_1 in X.columns and feature_2 in X.columns:
                interaction_name = f"{feature_1}_x_{feature_2}"
                if interaction_name not in X.columns:
                    X[interaction_name] = X[feature_1] * X[feature_2]

        # Tambahkan kolom yang hilang (jika ada)
        missing_cols = set(self.columns) - set(X.columns)
        for col in missing_cols:
            X[col] = 0  # Isi dengan nilai default

        # Urutkan kolom sesuai dengan data latih
        return X[self.columns]


Tidak ada kolom yang didrop

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [25]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [26]:
class FeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.q1 = None
        self.q3 = None
        self.iqr = None

    def fit(self, X, y=None):
        # Pastikan input adalah DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("FeatureScaler hanya mendukung input pandas DataFrame.")

        # Hitung Q1, Q3, dan IQR untuk setiap kolom numerik
        self.q1 = X.quantile(0.25)
        self.q3 = X.quantile(0.75)
        self.iqr = self.q3 - self.q1
        return self

    def transform(self, X):
        # Pastikan input adalah DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("FeatureScaler hanya mendukung input pandas DataFrame.")

        # Terapkan scaling hanya pada kolom numerik
        X_scaled = (X - self.q1) / self.iqr.replace(0, 1)  # Hindari pembagian dengan nol
        return X_scaled


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

Kami tidak melakukan encoding karena beberapa kolom sudah kami drop dan juga kami sudah menerapkan labelling juga pada feature selection sehingga dapat dianggap encoding.

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [27]:
class SMOTEWrapper:
    def __init__(self, random_state=42):
        self.smote = SMOTE(random_state=random_state)

    def fit_resample(self, X, y):
        # Jika X adalah DataFrame, simpan kolom dan indeks
        if isinstance(X, pd.DataFrame):
            columns = X.columns
            X_resampled, y_resampled = self.smote.fit_resample(X, y)
            # Kembalikan hasil sebagai DataFrame
            return pd.DataFrame(X_resampled, columns=columns), pd.Series(y_resampled)
        else:
            # Jika X bukan DataFrame, hanya kembalikan hasil
            return self.smote.fit_resample(X, y)


# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [28]:
print("Kolom di train_set:", train_set.columns)
print("Kolom di val_set:", val_set.columns)


Kolom di train_set: Index(['id', 'URLLength', 'DomainLength', 'IsDomainIP', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LargestLineLength', 'HasTitle',
       'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots',
       'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription',
       'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet',
       'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay',
       'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
       'NoOfSelfRef', 'NoOfEmptyRef', 'label'],
      dtype='object')
Kolom di val_set: Index(['id', 'URLLength'

In [29]:
# Preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('remove_duplicates', RemoveDuplicatesTransformer()),  
    ('imputation', ImputationTransformer()),               
    ('outlier_handling', OutlierHandlerTransformer()),      
    ('feature_engineering', UnifiedFeatureEngineeringTransformer(
        important_features={'IsHTTPS', 'HasTitle', 'HasCopyrightInfo', 'NoOfImage', 'HasSocialNet'},
        low_corr_threshold=0.2,
        high_corr_threshold=0.9,
        bins=5
    )),                                                    
    ('scaler', FeatureScaler())                             
])

# Preprocessing data latih
X_train = train_set.drop(columns=['label', 'id'])
y_train = train_set['label']
X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train, y_train)

# Simpan kolom fitur dari data latih
if isinstance(X_train_preprocessed, pd.DataFrame):
    feature_columns = X_train_preprocessed.columns
else:
    feature_columns = range(X_train_preprocessed.shape[1])

# Preprocessing data validasi
X_val = val_set.drop(columns=['label', 'id'])
y_val = val_set['label']
X_val_preprocessed = preprocessing_pipeline.transform(X_val)

# Sinkronisasi kolom pada data validasi
missing_cols = set(feature_columns) - set(X_val_preprocessed.columns)
for col in missing_cols:
    X_val_preprocessed[col] = 0

extra_cols = set(X_val_preprocessed.columns) - set(feature_columns)
X_val_preprocessed = X_val_preprocessed.drop(columns=extra_cols)

# Debugging bentuk akhir
print("Shape X_train_preprocessed:", X_train_preprocessed.shape)
print("Shape X_val_preprocessed:", X_val_preprocessed.shape)

# Resampling dengan SMOTE
X_train_resampled, y_train_resampled = SMOTEWrapper().fit_resample(X_train_preprocessed, y_train)

# Jika DataFrame diperlukan setelah SMOTE
if isinstance(X_train_preprocessed, pd.DataFrame):
    X_train_resampled = pd.DataFrame(X_train_resampled, columns=feature_columns)

print("Shape X_train_resampled:", X_train_resampled.shape)
print("Shape X_val_preprocessed (after syncing features):", X_val_preprocessed.shape)


Tidak ada duplikasi ditemukan.


/Users/attarlisan/Documents/Tubes2_AI/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/attarlisan/Documents/Tubes2_AI/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Tidak ada duplikasi ditemukan.
Shape X_train_preprocessed: (22464, 11)
Shape X_val_preprocessed: (5617, 11)
Shape X_train_resampled: (41464, 11)
Shape X_val_preprocessed (after syncing features): (5617, 11)


/Users/attarlisan/Documents/Tubes2_AI/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [30]:
import numpy as np
from collections import Counter

class KNearestNeighbors:
    def __init__(self, k=3, metric='euclidean', p=2):
        self.k = k
        self.metric = metric
        self.p = p
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        if self.k > len(self.X_train):
            raise ValueError("Nilai k tidak boleh lebih besar dari jumlah sampel training.")
        return self

    def predict(self, X):
        X = np.array(X)
        predictions = [self._predict_one(x) for x in X]
        return np.array(predictions)

    def _predict_one(self, x):
        distances = self._compute_distances(x)
        k_indices = np.argsort(distances)[:self.k]
        k_labels = self.y_train[k_indices]
        most_common = Counter(k_labels).most_common(1)
        return most_common[0][0]

    def _compute_distances(self, x):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        elif self.metric == 'minkowski':
            return np.sum(np.abs(self.X_train - x) ** self.p, axis=1) ** (1 / self.p)
        else:
            raise ValueError("Metrik tidak valid. Gunakan 'euclidean', 'manhattan', atau 'minkowski'.")


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

print("Shape X_train_resampled:", X_train_resampled.shape)
print("Shape X_val_preprocessed:", X_val_preprocessed.shape)


knn_sklearn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=3)
knn_sklearn.fit(X_train_resampled, y_train_resampled)
y_pred_knn_sklearn = knn_sklearn.predict(X_val_preprocessed)

print("KNN Scikit-Learn Accuracy:", accuracy_score(y_val, y_pred_knn_sklearn))
print("KNN Scikit-Learn Classification Report:\n", classification_report(y_val, y_pred_knn_sklearn))

Shape X_train_resampled: (41464, 11)
Shape X_val_preprocessed: (5617, 11)
KNN Scikit-Learn Accuracy: 0.9503293573081716
KNN Scikit-Learn Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.83      0.72       420
           1       0.99      0.96      0.97      5197

    accuracy                           0.95      5617
   macro avg       0.81      0.90      0.84      5617
weighted avg       0.96      0.95      0.95      5617



## B. Naive Bayes

In [32]:
class GaussianNaiveBayes:
    def __init__(self):
        self.classes = None
        self.class_priors = {}
        self.class_stats = {}

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.classes = np.unique(y)
        
        for cls in self.classes:
            X_cls = X[y == cls]
            self.class_priors[cls] = len(X_cls) / len(y)
            self.class_stats[cls] = {
                "mean": np.mean(X_cls, axis=0),
                "var": np.var(X_cls, axis=0) + 1e-9  
            }
        return self

    def predict(self, X):
        X = np.array(X)
        predictions = [self._predict_one(x) for x in X]
        return np.array(predictions)

    def _predict_one(self, x):
        posteriors = []
        for cls in self.classes:
            prior = np.log(self.class_priors[cls])
            likelihood = -0.5 * np.sum(
                np.log(2 * np.pi * self.class_stats[cls]["var"])
            ) - 0.5 * np.sum(
                ((x - self.class_stats[cls]["mean"]) ** 2) / self.class_stats[cls]["var"]
            )
            posteriors.append(prior + likelihood)
        return self.classes[np.argmax(posteriors)]


In [33]:
from sklearn.naive_bayes import GaussianNB

# Scikit-learn Naive Bayes
gnb_sklearn = GaussianNB()
gnb_sklearn.fit(X_train_resampled, y_train_resampled)
y_pred_gnb_sklearn = gnb_sklearn.predict(X_val_preprocessed)

print("Gaussian Naive Bayes Scikit-Learn Accuracy:", accuracy_score(y_val, y_pred_gnb_sklearn))
print("Gaussian Naive Bayes Scikit-Learn Classification Report:\n", classification_report(y_val, y_pred_gnb_sklearn))


Gaussian Naive Bayes Scikit-Learn Accuracy: 0.9611892469289657
Gaussian Naive Bayes Scikit-Learn Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.50      0.66       420
           1       0.96      1.00      0.98      5197

    accuracy                           0.96      5617
   macro avg       0.97      0.75      0.82      5617
weighted avg       0.96      0.96      0.96      5617



In [34]:
train_set.shape

(22464, 49)

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [36]:
test_path = '/Users/attarlisan/Documents/Tubes2_AI/.csv/test.csv'
test_set = pd.read_csv(test_path)

# Proses test set menggunakan pipeline
X_test = test_set.drop(columns=['id'])  # Hilangkan kolom 'id'
X_test_preprocessed = preprocessing_pipeline.transform(X_test)


Tidak ada duplikasi ditemukan.


/Users/attarlisan/Documents/Tubes2_AI/.venv/lib/python3.13/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [38]:
# Prediksi menggunakan model KNN
y_test_pred = knn_sklearn.predict(X_test_preprocessed)


In [39]:
# Membuat DataFrame untuk submission
submission = pd.DataFrame({
    'id': test_set['id'],  # Kolom 'id' dari test set
    'label': y_test_pred   # Prediksi label
})

# Simpan file submission
submission_path = '/Users/attarlisan/Documents/Tubes2_AI/submission.csv'
submission.to_csv(submission_path, index=False)

print(f"Submission file saved to: {submission_path}")


Submission file saved to: /Users/attarlisan/Documents/Tubes2_AI/submission.csv


# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

- Dari model yang sudah dibuat, terlihat baik dari KNN menggunakan euclidean, manhattan, maupun minkowski bahwa model lebih baik dalam memprediksi class 1 (legitimate) dibandingkan class 0 (phishing). Terlihat bahwa akurasi untuk presisi untuk class 1 /legit adalah 90% ke atas untuk semua model sedangkan akurasi untuk class 0/ phishing adalah di bawah 60%. Hal ini kemungkinan dapat terjadi karena data yang tidak seimbang, terlihat bahwa frekuensi class legit adalah 5197 dan frekuensi class legit hanya 420 (sampled) sehingga terhadap bias untuk majority class yang menyebabkan false positive pada minority class. Untuk Naive Bayes sendiri, performa untuk memprediksi class legit dan class phishing serupa yaitu di atas 95%, akan tetapi recall untuk ss 0 cukup rendah. Hal ini dapat terjadi karena Naive Bayes mempertimbangkan seluruh dataset, sehingga tidak terlalu terganggu oleh ketidakseimbangan data dalam memprediksi kelas minoritas .
- Dari Kedua model yang dibuat dan 3 distance yang berbeda untuk KNN, maka model terbaik adalah KNN dengan distance manhattan dan Naive Bayes dengan akurasi yang serupa, yaitu 95-96%
- Untuk data yang hilang, jika data tersebut terlalu banyak; melebihi 49% maka kolom tersebut dihapus secara keseluruhan karena tidak begitu bermakna baik jika diimputasi ataupun dihapus, untuk kolom kategorikal juga kami drop. Sedangkan untuk missing value di kolom lain dilakukan imputasi karena jika missing value dihapus maka akan ada informasi penting yang hilang. Imputasi dilakukan untuk menjaga ukuran dataset dan mempertahankan distribusi fitur, sehingga model memiliki akses ke lebih banyak data. Namun, metode imputasi (mean, median, mode, dll.) harus sesuai dengan sifat data yang hilang dan jenis fiturnya.
- Scaling dengan IQR Normalization seperti yang kami lakukan berguna untuk beberapa hal, terutama untuk KNN yaitu untuk membuat perhitungan jarak menjadi lebih adil dan akurat, menangani outlier secara lebih baik, serta untuk mengatasi ketidakseimbangan fitur.